In [1]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,PostbackTemplateAction,MessageTemplateAction,URITemplateAction
)

app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# create a instance for line
# line_bot_api 用來處理消息
line_bot_api = LineBotApi('YOUR_CHANNEL_ACCESS_TOKEN')

# 用來接收外部消息
handler = WebhookHandler('YOUR_CHANNEL_SECRET')

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body 驗證封包是否來自Line
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'
    
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        thumbnail_image_url='https://1db310cb.ap.ngrok.io/images/eddie.jpg',
        title='Menu',
        text='Please select',
        actions=[
            PostbackTemplateAction(
                label='postback',
                text='同時也可以讓用戶發送文字訊息',
                data='action=buy&itemid=1'
            ),
            MessageTemplateAction(
                label='message',
                text='資策會聊天機器人'
            ),
            URITemplateAction(
                label='uri',
                uri='https://www.youtube.com/watch?v=uauzw00I0wY'
            )
        ]
    )
)
    
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        buttons_template_message)

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [01/May/2017 12:02:22] "POST / HTTP/1.1" 200 -
